In [3]:
print("OKAY")

OKAY


In [52]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import os
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [5]:
load_dotenv()

True

In [6]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = "gcp-starter"

In [7]:
def data_loader(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [8]:
extracted_data= data_loader(r"C:\Users\furkanbaba\Desktop\furkanbaba\coding\Rag-Project\medical_chatbot\data")

In [9]:
#extracted_data

In [10]:
def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
    text_chunks = splitter.split_documents(extracted_data)
    return text_chunks


In [11]:
text_chunks = text_splitter(extracted_data)
print("length of text:", len(text_chunks))

length of text: 3367


In [12]:
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_embeddings()

In [14]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={}


In [15]:
query_results = embeddings.embed_query("HI, how are you feeling today ?")
print("length:",len(query_results))

length: 384


In [16]:
query_results

[-0.04761326313018799,
 0.012561398558318615,
 0.08829054236412048,
 0.016965650022029877,
 0.04545752331614494,
 -0.04732634872198105,
 0.09706729650497437,
 -0.01631682924926281,
 -0.05710430443286896,
 -0.007524485699832439,
 -0.04716635122895241,
 0.011600297875702381,
 -0.017519285902380943,
 0.019553260877728462,
 0.03404289856553078,
 -0.003682906273752451,
 0.004642177373170853,
 -0.09220511466264725,
 -0.12892168760299683,
 0.08274782449007034,
 -0.08603700250387192,
 -0.013935116119682789,
 -0.05553831160068512,
 0.09202856570482254,
 0.01082191988825798,
 0.052058979868888855,
 0.0026824248488992453,
 0.009897642768919468,
 -0.011292709037661552,
 -0.11285684257745743,
 -0.04507773369550705,
 0.026827633380889893,
 -0.022926315665245056,
 -0.022284774109721184,
 -0.004771487321704626,
 0.025244969874620438,
 -0.0174946840852499,
 -0.16253238916397095,
 -0.004957687109708786,
 -0.011241892352700233,
 0.03618461638689041,
 -0.024022530764341354,
 0.017015555873513222,
 -0.0037

In [17]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index('medical-chatbot') 

In [18]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.10387,
 'namespaces': {'': {'vector_count': 10387}},
 'total_vector_count': 10387}

In [50]:
index_name="medical-chatbot"
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

KeyboardInterrupt: 

In [97]:
query = "What is  acne?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is


In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [77]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [76]:
prompt = ChatPromptTemplate.from_template(prompt_template)

In [55]:
llm=CTransformers(model=r"C:\Users\furkanbaba\Desktop\furkanbaba\coding\Rag-Project\medical_chatbot\model\llama-2-7b-chat.ggmlv3.q4_0 (1).bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [148]:
retriever=docsearch.as_retriever(search_kwargs={'k': 2})

In [157]:
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)  

In [158]:
qa=load_qa_chain(llm=llm,chain_type="stuff",prompt=PROMPT)

In [135]:
print(type(docs[0]))

<class 'langchain_core.documents.base.Document'>


In [159]:
from langchain.schema import Document

In [160]:
def retrieve_query(query, k=2):
    matching_results = vectorstore.similarity_search(query, k=k)
    if matching_results:
        best_match_content = matching_results[0].page_content
        best_document = Document(page_content=best_match_content)
        return [best_document] 
    else:
        return []

In [161]:
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=qa.run(input_documents=doc_search,question=query)
    return response

In [162]:
our_query = "what is sore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='treatable and so pose little risk.KEY TERMS\nCanker sore —A painful sore inside the mouth.\nCold sore —A small blister on the lips or face,\ncaused by a virus. Also called a fever blister.\nEpidural space —The space surrounding the spinal\nfluid sac.\nMalignant hyperthermia —A type of reaction\n(probably with a genetic basis) that can occur dur-ing general anesthesia in which the patient experi-ences a high fever, the muscles become rigid, andthe heart rate and blood pressure fluctuate.', metadata={})]
A canker sore is a type of painful sore that occurs inside the mouth. It is typically small and round, with a white or yellowish center surrounded by a red ring. Canker sores are not contagious and do not pose a risk to one's health, unlike cold sores which are caused by a virus and can be spread through close contact with an infected person.
